In [2]:
import pandas as pd
from embeds import multi_inner_align, standardize
import sys
sys.path.append('..')
from rca.rca import run_rca

In [5]:
ft_baseline = pd.read_csv('../../data/embeds/fastText_CommonCrawl.csv', index_col=0)
Llama_3_1B = pd.read_csv('../../data/llms/Llama-3.2-1B.csv', index_col=0)

# Comparing
to_compare = {
    'ft_baseline': ft_baseline,
    'Llama_3.2_1B': Llama_3_1B
}

# Aligning for fair comparison
to_compare = dict(zip(to_compare.keys(), multi_inner_align(to_compare.values())))

# Standardizing
to_compare = {name: standardize(embed) for name, embed in to_compare.items()}

# Loading norm data
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, low_memory=False, compression='zip')
norms_meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col='norm')


[embed.shape for embed in to_compare.values()]

[(44450, 300), (44450, 2048)]

In [7]:
results = run_rca(to_compare, norms, norms_meta, n_jobs=10)
results

  0%|          | 0/2 [00:00<?, ?it/s]

ft_baseline:   0%|          | 0/291 [00:00<?, ?it/s]

/Users/zhussain/miniconda3/envs/psychProbing_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/miniconda3/envs/psychProbing_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/miniconda3/envs/psychProbing_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/miniconda3/envs/psychProbing_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/miniconda3/envs/psychProbing_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

                               norm  train_n   r2_mean     r2_sd check
217                social_des_britz      384  0.878681  0.017609  pass
216                   valence_britz      384  0.876765  0.019279  pass
279              thought_vanarsdall      960  0.861191  0.009895  pass
281               person_vanarsdall      960  0.853676  0.012779  pass
267            likableness_anderson      405  0.851399  0.021847  pass
211           tabooness_janschewitz      359  0.843650  0.057847  pass
282                goals_vanarsdall      960  0.839665  0.028578  pass
11   context_diversity_blog_gimenes    32329  0.836941  0.006665  pass
0                    frequency_lund    28423  0.835884  0.015643  pass
4            frequency_blog_gimenes    32329  0.834497  0.007229  pass


Llama_3.2_1B:   0%|          | 0/291 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Adding norm category
results['norm_category'] = (
    results['norm']
    .apply(lambda norm: norms_meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

results_avg = (
    results[['norm_category', 'embed', 'r2_mean']]
    .groupby(['norm_category', 'embed'], as_index=False).median()
    .dropna()
)

results_avg_piv = results_avg.pivot(columns='embed', index='norm_category', values='r2_mean')
results_avg_piv.round(2)

In [ ]:
# Finding the top-performing fmri_text_denoise
sorted_overall = results_avg_piv.mean().sort_values(ascending=False)
sorted_overall

In [ ]:
## Saving
top_performer = None
top_performer.to_csv('../../data/embeds/Llama_X_XB.csv')